<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [63]:
##### Your Code Here #####
import numpy as np

np.random.seed(812)

# hours studying, hours sleep
X = np.array(([0,0,1],
              [0,1,1],
              [1,0,1],
              [0,1,0],
             [1,0,0],
             [1,1,1],
             [0,0,0]), dtype=int)

# Exam Scores
y = np.array(([0],
              [1],
              [1],
             [1],
             [1],
             [0],
             [0]), dtype=int)

In [64]:
# I want activations that correspond to negative weights to be lower
# and activations that correspond to positive weights to be higher

class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

        # Initial Weights
        # 2x3 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # How much of that "far off" can explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

In [65]:
nn = NeuralNetwork()
cost = []
for i in range(1000):
    cost.append(np.mean(np.square(y - nn.feed_forward(X))))
    nn.train(X, y)
print("Predicted Output: \n" + str(nn.feed_forward(X))) 
print("Loss: \n" + str(np.mean(np.square(y - nn.feed_forward(X)))))

Predicted Output: 
[[0.00786547]
 [0.95517652]
 [0.95406274]
 [0.95202678]
 [0.95352858]
 [0.05432688]
 [0.08329347]]
Loss: 
0.0026473537692100974


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [66]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [67]:
# input image dimensions
img_rows, img_cols = 28, 28

In [68]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [69]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [70]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [71]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [72]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [73]:
##### Your Code Here #####
weights1 = np.random.rand(784, 100)
bias1 = np.random.random()

weights2 = np.random.rand(100, 1)
bias2 = np.random.random()

def output(X):
    weighted_sum1 = np.dot(X, weights1) + bias1
    activated1 = sigmoid(weighted_sum1)
    weighted_sum2 = np.dot(activated1, weights2) + bias2
    activated2 = sigmoid(weighted_sum2)
    return (activated1, activated2)

In [74]:
# x_train is too large, so I'll split it up.
X = x_train[0:1000]
y = y_train[0:1000]

In [75]:
for _ in range(1000):
    (hidden, out) = output(X)
    
    # print("hidden", hidden.shape)
    # print("out", out.shape)
    
    out_error = y - out
    out_delta = out_error * sigmoidPrime(out)
    
    # print("out_error", out_error.shape)
    # print("out_delta", out_delta.shape)
    
    z2_error = np.dot(out_delta, weights2.T)
    z2_delta = z2_error * sigmoidPrime(hidden)
    
    # print(np.mean(out_delta))
    
    weights1 += np.dot(X.T, z2_delta)
    bias1 += np.mean(z2_error)
    
    weights2 += np.dot(hidden.T, out_delta)
    bias2 += np.mean(out_error)

print(np.mean(out_error))

NameError: name 'sigmoid' is not defined

In [56]:
NN = Neural_Network()
T = trainer(NN)
T.train(x_train,y_train)

ValueError: shapes (60000,784) and (3,4) not aligned: 784 (dim 1) != 3 (dim 0)

In [ ]:
print("Predicted Output: \n" + str(NN.forward(X))) 
print("Loss: \n" + str(np.mean(np.square(y - NN.forward(X))))) # mean sum squared loss

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?